# Preprocessing

## 1. Introduction
This notebook implements the complete pipeline for:
1. Loading and preprocessing Empatica E4 wearable data
2. Feature engineering for sleep stage classification
3. Quality control and data cleaning
4. Preparing data for models

## 2. Setup and Configuration

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
from tqdm import tqdm
import neurokit2 as nk
from scipy import signal
from scipy.interpolate import interp1d
from scipy.stats import skew
from collections import Counter
import warnings
from multiprocessing import Pool, cpu_count

# Configuration
DATA_FOLDER = "data/datasets"
FEATURES_DIR = "data/features_df"
INFO_FILE = "data/participant_info.csv"
QUALITY_FILE = "data/quality_scores_per_subject.csv"
THRESHOLD = 0.2  # Quality threshold

# Set display options
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 20)
pd.set_option("display.width", 1000)

# Ignore warnings
warnings.filterwarnings("ignore")

## 3. Data Loading and Preprocessing

### 3.1 Core Preprocessing Functions

In [ ]:
def preprocess_BVP(bvp):
    """Preprocess BVP signal with Chebyshev filter"""
    sos = signal.cheby2(N=10, rs=40, Wn=[0.5, 15], btype="bandpass", fs=64, output="sos")
    return signal.sosfilt(sos, bvp)

def preprocess_ACC(acc, fs=32):
    """Bandpass filter accelerometer data"""
    sos = signal.butter(N=3, Wn=[3, 10], btype="bp", fs=fs, output="sos")
    return signal.sosfilt(sos, acc)

def preprocess_EDA(eda, fs=4):
    """Detrend and filter EDA signal"""
    # Detrend in 5-second segments
    detrended = []
    for i in range(len(eda)//20):
        segment = eda[i*20:(i+1)*20]
        m, b = np.polyfit(np.arange(20), segment, 1)
        detrended.append(segment - (m*np.arange(20) + b)
    
    # Filter
    sos = signal.butter(N=3, Wn=0.7, fs=fs, output="sos")
    return signal.sosfilt(sos, np.concatenate(detrended))

def preprocess_ALL_SIGNALS(df):
    """Preprocess all signals in a dataframe"""
    processed_df = df.copy()
    
    # Process each signal
    processed_df["BVP"] = preprocess_BVP(df["BVP"])
    processed_df["ACC_X"] = preprocess_ACC(df["ACC_X"])
    processed_df["ACC_Y"] = preprocess_ACC(df["ACC_Y"])
    processed_df["ACC_Z"] = preprocess_ACC(df["ACC_Z"])
    processed_df["EDA"] = preprocess_EDA(df["EDA"])
    
    # Process timestamps
    ts = df['TIMESTAMP'].to_numpy()
    ts = ts - ts[0]
    processed_df["TIMESTAMP_COSINE"] = circadian_cosine(ts)
    processed_df["TIMESTAMP_DECAY"] = circadian_decay(ts)
    processed_df["TIMESTAMP_LINEAR"] = circadian_linear(ts)
    
    return processed_df

### 3.2 Circadian Rhythm Features

In [ ]:
def circadian_cosine(ts, samp_freq=64):
    """Calculate circadian cosine feature"""
    len_vec = (ts[-1] - ts[0]) * samp_freq
    return np.sin((2 * np.pi / (len_vec * 2)) * np.arange(len(ts)))

def circadian_decay(ts, samp_freq=64):
    """Calculate circadian decay feature"""
    len_vec = (ts[-1] - ts[0]) * samp_freq
    k = np.log(0.01) / len_vec
    return np.exp(k * np.arange(len(ts)))

def circadian_linear(ts, samp_freq=64):
    """Calculate circadian linear feature"""
    len_vec = (ts[-1] - ts[0]) * samp_freq
    return np.arange(len(ts)) / len_vec

## 4. Feature Extraction

### 4.1 Signal Quality Assessment

In [ ]:
def exclude_signal(segment_df):
    """Identify if a segment should be labeled as artifact"""
    bvp = segment_df.BVP.to_numpy()
    bvp = bvp - np.mean(bvp)
    
    # Filter and calculate SNR
    b, a = signal.butter(2, [0.5/32, 15/32], btype="band")
    filtered = signal.filtfilt(b, a, bvp)
    snr_db = 10 * np.log10(np.mean(filtered**2) / np.mean((bvp-filtered)**2))
    
    # Calculate activity index
    acc = np.sqrt(segment_df[['ACC_X','ACC_Y','ACC_Z']].mean(axis=1))
    acc_std = np.std(acc)
    
    # Rule-based artifact detection
    if (acc_std >= 0.4125/2) or (snr_db < 10) or \
       (np.max(bvp) > 500) or (np.min(bvp) < -500):
        return 1
    return 0

### 4.2 HRV Feature Extraction

In [ ]:
HRV_FEATURES = [
    "HRV_MeanNN", "HRV_SDNN", "HRV_RMSSD", "HRV_pNN50",
    "HRV_LF", "HRV_HF", "HRV_LFHF", "HRV_SD1", "HRV_SD2"
]

def extract_hrv_features(segment_df):
    """Extract HRV features using NeuroKit2"""
    try:
        signals, info = nk.ppg_process(segment_df.BVP, sampling_rate=64)
        results = nk.ppg_analyze(signals, sampling_rate=64)
        return {f: results[f].iloc[0] for f in HRV_FEATURES}, 0
    except:
        return {f: np.nan for f in HRV_FEATURES}, -1

### 4.3 Accelerometer Features

In [ ]:
def extract_acc_features(segment_df):
    """Extract accelerometer features"""
    acc_x = segment_df.ACC_X.to_numpy()
    acc_y = segment_df.ACC_Y.to_numpy()
    acc_z = segment_df.ACC_Z.to_numpy()
    acc_mag = np.sqrt(acc_x**2 + acc_y**2 + acc_z**2)
    
    # Trimmed statistics (ignore top/bottom 10%)
    acc_trimmed = acc_mag[(acc_mag > np.quantile(acc_mag, 0.1)) & 
                         (acc_mag < np.quantile(acc_mag, 0.9))]
    
    return {
        "ACC_mean": np.mean(acc_trimmed),
        "ACC_max": np.max(acc_trimmed),
        "ACC_iqr": np.percentile(acc_trimmed, 75) - np.percentile(acc_trimmed, 25),
        "ACC_std": np.std(acc_mag)
    }

### 4.4 Complete Feature Extraction

In [ ]:
def extract_domain_features(sid, data_folder, save_folder_dir, segment_seconds=30):
    """Extract all features for one subject"""
    # Load and preprocess data
    df = pd.read_csv(f"{data_folder}/{sid}_whole_df.csv")
    df = preprocess_ALL_SIGNALS(df)
    
    # Initialize feature storage
    epoch_length = segment_seconds * 64
    num_segments = len(df) // epoch_length
    features = []
    
    # Process each epoch
    for i in tqdm(range(num_segments), desc=f"Processing {sid}"):
        segment = df.iloc[i*epoch_length:(i+1)*epoch_length]
        
        # Extract features
        hrv_feats, _ = extract_hrv_features(segment)
        acc_feats = extract_acc_features(segment)
        
        # Combine all features
        epoch_feats = {**hrv_feats, **acc_feats}
        epoch_feats['artifact'] = exclude_signal(segment)
        epoch_feats['sid'] = sid
        features.append(epoch_feats)
    
    # Save features
    features_df = pd.DataFrame(features)
    features_df.to_csv(f"{save_folder_dir}/{sid}_domain_features_df.csv", index=False)
    return features_df

## 5. Quality Control and Data Preparation

### 5.1 Calculate Quality Scores

In [ ]:
def calculate_quality_scores(feature_path, output_path):
    """Calculate quality scores for all subjects"""
    files = [f for f in os.listdir(feature_path) if f.endswith('_domain_features_df.csv')]
    results = []
    
    for file in tqdm(files, desc="Calculating quality scores"):
        sid = file.split('_')[0]
        df = pd.read_csv(f"{feature_path}/{file}")
        perc_excluded = df['artifact'].mean()
        results.append({
            'sid': sid,
            'total_segments': len(df),
            'num_excludes': df['artifact'].sum(),
            'percentage_excludes': perc_excluded
        })
    
    quality_df = pd.DataFrame(results)
    quality_df.to_csv(output_path, index=False)
    return quality_df

### 5.2 Prepare Modeling Data

In [ ]:
def prepare_modeling_data(threshold=0.2):
    """Prepare final dataset for modeling"""
    # Load quality scores and filter subjects
    quality_df = pd.read_csv(QUALITY_FILE)
    good_sids = quality_df[quality_df.percentage_excludes < threshold].sid.tolist()
    
    # Load and concatenate all feature files
    dfs = []
    for sid in tqdm(good_sids, desc="Loading feature files"):
        df = pd.read_csv(f"{FEATURES_DIR}/{sid}_domain_features_df.csv")
        dfs.append(df)
    
    full_df = pd.concat(dfs)
    
    # Clean data
    full_df = full_df.replace([np.inf, -np.inf], np.nan)
    full_df = full_df.dropna(axis=1, thresh=0.8*len(full_df))  # Drop columns with >20% NA
    full_df = full_df.dropna()  # Drop remaining rows with NA
    
    # Save cleaned data
    os.makedirs("data/processed", exist_ok=True)
    full_df.to_csv("data/processed/clean_df.csv", index=False)
    
    # Get final feature list
    features = [col for col in full_df.columns if col not in 
               ['sid', 'Sleep_Stage', 'artifact']]
    
    return full_df, features, good_sids

## 6. Running the Complete Pipeline

### 6.1 Process All Subjects

In [ ]:
def process_all_subjects():
    """Run complete processing pipeline"""
    # Create directories
    os.makedirs(FEATURES_DIR, exist_ok=True)
    
    # Get list of subjects to process
    sid_list = [f.split('_')[0] for f in os.listdir(DATA_FOLDER) 
               if f.endswith('_whole_df.csv')]
    
    # Process each subject
    for sid in tqdm(sid_list, desc="Processing subjects"):
        if not os.path.exists(f"{FEATURES_DIR}/{sid}_domain_features_df.csv"):
            try:
                extract_domain_features(
                    sid=sid,
                    data_folder=DATA_FOLDER,
                    save_folder_dir=FEATURES_DIR,
                    segment_seconds=30
                )
            except Exception as e:
                print(f"Error processing {sid}: {str(e)}")
    
    # Calculate quality scores
    calculate_quality_scores(FEATURES_DIR, QUALITY_FILE)
    
    # Prepare final modeling dataset
    clean_df, features, good_sids = prepare_modeling_data(threshold=THRESHOLD)
    
    print("\nPipeline completed successfully!")
    print(f"Final dataset contains {len(clean_df)} epochs from {len(good_sids)} subjects")
    print(f"Number of features: {len(features)}")
    
    return clean_df, features, good_sids

# Uncomment to run the complete pipeline
# clean_df, features, good_sids = process_all_subjects()